<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/1_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final preprocessing of full data set for 4 question types (! STRONG threshold here!)

In [1]:
from google.colab import userdata, drive
import os

In [2]:
token = userdata.get('GITHUB')
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

Cloning into 'NLP2025_CQG'...
remote: Enumerating objects: 976, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 976 (delta 157), reused 88 (delta 59), pack-reused 737 (from 2)
Receiving objects: 100% (976/976), 46.18 MiB | 23.32 MiB/s, done.
Resolving deltas: 100% (509/509), done.
Updating files: 100% (95/95), done.


In [3]:
os.chdir("NLP2025_CQG")
!ls

1_Information_preprocessing.md	      Doc
1_Preprocessing.ipynb		      Evaluation
2_Baseline_Generation.ipynb	      INFORMATION.md
2_Information_Baseline_Generation.md  LICENSE
3_Evaluation.ipynb		      Logs
4_Finetuned_Generation.ipynb	      README.md
5_Evaluation_Analytics.ipynb	      requirements.txt
Data				      Training
Development			      Utils


In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import spacy
import nltk

nltk.download('framenet_v17')
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package framenet_v17 to /root/nltk_data...
[nltk_data]   Unzipping corpora/framenet_v17.zip.


In [ ]:
chunk = "train_chunk_III"

filtered_df = pd.read_csv(
    f'/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/length_filtered/filtered_{chunk}.csv'
)

### lentgh filtering

In [ ]:
df = pd.read_csv(
    os.path.join(os.getcwd(), f"Data/Raw/SocraticQ/{chunk}.csv"),
    names=["category", "context", "question"]
)

df["context_token_len"] = df["context"].apply(lambda text: len(nlp(text)))
filtered_df = df[df["context_token_len"] >= 25].copy()

print(f"Total rows: {len(df)}")
print(f"Rows after filtering: {len(filtered_df)}")

In [ ]:
filtered_df.to_csv(f'/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/length_filtered/filtered_{chunk}.csv', index=False)

### 1. cause to effect

In [ ]:
from nltk.corpus import framenet as fn

def get_causal_verbs_from_framenet():
    causal_frame_names = [
        "Causation", "Cause_change", "Cause_change_of_position_on_a_scale",
        "Cause_motion", "Cause_to_amalgamate", "Cause_to_start", "Cause_to_make_progress",
        "Causation_scenario", "Cause_to_end", "Cause_to_resume",
        "Cause_to_continue", "Cause_change_of_consistency","Cause_expansion","Cause_impact"
    ]

    causal_verbs = set()
    for frame_name in causal_frame_names:
        try:
            frame = fn.frame_by_name(frame_name)
            for lu in frame.lexUnit.values():
                if '.v' in lu['name']:  # Only verbs
                    causal_verbs.add(lu['name'].split('.')[0])
        except Exception as e:
            print(f"Error loading frame '{frame_name}': {e}")

    return causal_verbs


causal_meta_terms = {"generalisation", "implies", "entail", "necessitate", "follow from", "inference"}
alternative_factor_terms = {"factor", "interfere", "influence", "affect", "contribute", "complicate"}


def detect_cause_to_effect(sentence):
    doc = nlp(sentence)
    explanations = []
    score = 0

    causal_verbs = get_causal_verbs_from_framenet()

    has_condition = any(tok.dep_ == "mark" and tok.text.lower() in {"if", "when"} for tok in doc)
    if has_condition:
        explanations.append("✓ Conditional clause detected (e.g., 'if', 'when')")
        score += 3

    has_advcl = any(tok.dep_ == "advcl" for tok in doc)
    if has_advcl:
        explanations.append("✓ Adverbial clause (likely effect clause) detected")
        score += 2

    has_causal_verb_structure = False
    for tok in doc:
        if tok.lemma_ in causal_verbs and tok.pos_ == "VERB":
            subj = any(child.dep_ == "nsubj" for child in tok.children)
            obj = any(child.dep_ == "dobj" for child in tok.children)
            prep = any(child.dep_ == "prep" for child in tok.children)
            if subj or obj or prep:
                has_causal_verb_structure = True
                explanations.append(
                    f"✓ Verb '{tok.lemma_}' is listed in FrameNet under causal frames with subject/object/prep"
                )
                score += 3
                if subj: score += 0.5
                if obj: score += 0.5
                if prep: score += 0.5
                break

    if any(tok.lemma_ in causal_meta_terms for tok in doc):
      explanations.append("✓ Causal generalisation or implication term detected (e.g., 'implies', 'generalisation')")
      score += 1

    if any(tok.lemma_ in alternative_factor_terms for tok in doc):
      explanations.append("✓ Terms indicating alternative causes or interfering factors detected")
      score += 1

    is_causal = has_condition and has_advcl or has_causal_verb_structure
    if not is_causal:
        causal_phrases = ["result in", "lead to", "may cause", "because of", "due to","given rise to","resulting from", "stemming from", "driven by", "caused by", "attributed to", "stems from", "reason", "result of", "consequence of", "owning to", "thus", "so", "therefore", "hence"  "thereby"]
        if any(phrase in sentence.lower() for phrase in causal_phrases):
            explanations.append("✓ Phrase pattern matches known cause-to-effect trigger")
            score += 2

    score = min(score, 10)
    label = "Strong CauseToEffect" if score >= 7 else "Weak/Partial CauseToEffect" if score >= 4 else "Not CauseToEffect"
    return label, score, explanations

### 2. expert opinion

In [ ]:
def get_lexical_units_from_frames(frames):
    terms = set()
    for frame_name in frames:
        try:
            frame = fn.frame_by_name(frame_name)
            for lu in frame.lexUnit.values():
                if '.v' in lu['name']:
                    terms.add(lu['name'].split('.')[0])
        except Exception as e:
            print(f"Warning: Could not load frame '{frame_name}': {e}")
    return terms


expert_frames = [
    "Expertise", "Judgment_communication", "Opinion",
    "Authority", "Statement", "Certainty"
]
quote_frames = ["Statement", "Judgment_communication"]
clarity_frames = ["Reasoning"]
evidence_frames = ["Evidence", "Certainty", "Causation"]


expert_verbs = get_lexical_units_from_frames(expert_frames)
quote_verbs = get_lexical_units_from_frames(quote_frames)
clarity_terms = get_lexical_units_from_frames(clarity_frames)
evidence_terms = get_lexical_units_from_frames(evidence_frames)

def detect_expert_opinion(question):

    doc = nlp(question)
    score = 0
    explanations = []

    expert_titles = {"expert", "researcher", "scientist", "doctor", "analyst", "professor", "Dr."}

    implicit_expert_terms = {"study", "research", "evidence", "report", "findings", "scientific", "government", "official", "paper", "survey", "data"}
    comparison_cues = {"consistent", "align", "similar", "agree", "disagree", "corroborate", "conflict"}
    technical_request_verbs = {"define", "explain", "describe", "elaborate", "clarify"}
    assertion_verbs = {"assert", "affirm", "pronounce", "declare", "maintain", "claim", "state"}
    reference_terms = {"quote", "reference", "cite", "check", "verify", "source"}
    domain_terms = {"science", "scientific", "domain", "field", "discipline", "area", "sector"}


    for ent in doc.ents:
        if ent.label_ in {"PERSON", "ORG"}:
            if any(title in ent.text.lower() for title in expert_titles):
                explanations.append(f"✓ Expert entity detected: '{ent.text}'")
                score += 3
                break

    if any(tok.lemma_ in expert_verbs for tok in doc if tok.pos_ == "VERB"):
        explanations.append("✓ Detected expert-related verb from FrameNet")
        score += 2

    if any(tok.lemma_ in quote_verbs for tok in doc):
        explanations.append("✓ Quotation or claim verb found")
        score += 1

    if any(tok.lemma_ in clarity_terms for tok in doc):
        explanations.append("✓ Clarity/definition markers found")
        score += 1

    if any(tok.lemma_ in evidence_terms for tok in doc):
        explanations.append("✓ Evidence or support-related terms found")
        score += 2

    if any(tok.lemma_.lower() in implicit_expert_terms for tok in doc):
      explanations.append("✓ Implicit expert-related term detected (e.g., 'study', 'government')")
      score += 2

    if any(tok.lemma_.lower() in comparison_cues for tok in doc):
      explanations.append("✓ Cross-study comparison term detected (e.g., 'consistent', 'similar')")
      score += 0.5

    if any(tok.lemma_.lower() in technical_request_verbs for tok in doc):
      explanations.append("✓ Technical explanation request detected (e.g., 'define', 'explain')")
      score += 1

    if any(tok.dep_ == "attr" and tok.lemma_ == "expert" for tok in doc):
      explanations.append("✓ Predicate nominative indicating expertise detected (e.g., 'X is an expert')")
      score += 2

    if any(tok.lemma_.lower() in assertion_verbs for tok in doc):
      explanations.append("✓ Assertion or claim verb detected (e.g., 'assert', 'affirm')")
      score += 1

    if any(tok.lemma_.lower() in reference_terms for tok in doc):
      explanations.append("✓ Source/reference validation term detected (e.g., 'quote', 'reference')")
      score += 1

    if any(tok.lemma_.lower() in domain_terms for tok in doc):
      explanations.append("✓ Domain relevance indicator detected (e.g., 'science', 'domainD')")
      score += 1

    score = min(score, 10)
    label = "Strong Expert Opinion" if score >= 7 else "Weak/Partial Expert Opinion" if score >= 4 else "Not Expert Opinion"
    return label, score, explanations

### 3. Analogy detection

In [ ]:
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
analogy_synsets = [wn.synset('similar.a.01'), wn.synset('analogy.n.01'), wn.synset('compare.v.01')]

comparison_frames = ["Similarity"]
contrast_frames = ["Categorization"]
evidence_frames = ["Evidence", "Judgment_communication"]

comparison_verbs = get_lexical_units_from_frames(comparison_frames)
contrast_verbs = get_lexical_units_from_frames(contrast_frames)
evidence_verbs = get_lexical_units_from_frames(evidence_frames)

def is_semantically_analogical(token):
    token_synsets = wn.synsets(token.lemma_)
    for s in token_synsets:
        for analogy_syn in analogy_synsets:
            if s.path_similarity(analogy_syn) and s.path_similarity(analogy_syn) > 0.3:
                return True
    return False

analogy_context_cues = {"respect", "in which", "such that", "with regard to", "in terms of"}

analogy_force_cues = {"undermine", "weaken", "strengthen", "force of similarity", "degree of analogy"}

analogy_nouns = {"analogy", "comparison", "parallel", "similarity", "analogue"}

def detect_analogy_question(question):
    doc = nlp(question)
    score = 0
    explanations = []
    noun_chunks = list(doc.noun_chunks)

    if any(tok.lemma_ in comparison_verbs for tok in doc if tok.pos_ == "VERB"):
        explanations.append("✓ Comparison verb detected from FrameNet")
        score += 2.5

    entity_tokens = [tok for tok in doc if tok.pos_ in {"PROPN", "NOUN"}]
    if len(set(tok.lemma_ for tok in entity_tokens)) >= 2:
        explanations.append("✓ Contains at least two distinct concepts/entities")
        score += 1

    if any(tok.lemma_ in contrast_verbs for tok in doc if tok.pos_ == "VERB"):
        explanations.append("✓ Contrast or difference verb detected from FrameNet")
        score += 1

    if any(tok.lemma_ in evidence_verbs for tok in doc if tok.pos_ == "VERB"):
        explanations.append("✓ Evidence or justification verb found")
        score += 1

    if any(tok.tag_ == "MD" for tok in doc):
        score += 0.5

    if len(noun_chunks) >= 2 and any(tok.lemma_ in {"similar", "like", "as"} for tok in doc):
        explanations.append("✓ Two concepts compared with similarity cue (e.g., 'similar', 'like')")
        score += 3

    if any(tok.text.lower() == "if" for tok in doc):
        explanations.append("✓ Conditional structure suggesting hypothetical reasoning")
        score += 1

    if any(is_semantically_analogical(tok) for tok in doc if tok.pos_ in {"ADJ", "NOUN", "VERB"}):
        explanations.append("✓ Semantic similarity to analogy-related terms detected via WordNet")
        score += 2

    if any(tok.dep_ in {"prep", "relcl"} and tok.lemma_ in {"compare", "similar"} for tok in doc):
        explanations.append("✓ Syntactic cue of analogy (e.g., 'compared with', 'similar to')")
        score += 1

    if any(phrase in question.lower() for phrase in analogy_context_cues):
      explanations.append("✓ Contextual analogy marker detected (e.g., 'in which', 'such that')")
      score += 0.5

    if any(tok.lemma_ in analogy_force_cues for tok in doc):
      explanations.append("✓ Analogy evaluation term detected (e.g., 'undermine', 'strengthen')")
      score += 0.5

    if any(tok.lemma_ in analogy_nouns for tok in doc if tok.pos_ == "NOUN"):
      explanations.append("✓ Explicit analogy noun detected (e.g., 'analogy', 'comparison')")
      score += 2

    if any(tok.dep_ == "neg" for tok in doc):
      if any(tok.lemma_ in {"similar", "compare", "alike", "match"} for tok in doc):
          explanations.append("✓ Negated comparison detected (suggesting analogy breakdown)")
          score += 1

    score = min(score, 10)
    label = "Strong Analogy Question" if score >= 7 else "Weak/Partial Analogy Question" if score >= 4 else "Not Analogy Question"
    return label, score, explanations

### 4. Fear appeal

In [ ]:
def is_fear_related(token):
    syns = wn.synsets(token.lemma_)
    for s in syns:
        if any(s.path_similarity(wn.synset('danger.n.01')) or s.path_similarity(wn.synset('problem.n.01')) or
               s.path_similarity(wn.synset('fear.n.01')) or s.path_similarity(wn.synset('harm.n.01')) or
               s.path_similarity(wn.synset('threat.n.01')) for s in syns):
            return True
    return False


# ---- FrameNet Utility ----
def get_lexical_units_from_frames(frames):
    terms = set()
    for frame_name in frames:
        try:
            frame = fn.frame_by_name(frame_name)
            for lu in frame.lexUnit.values():
                if '.v' in lu['name']:
                    terms.add(lu['name'].split('.')[0])
        except:
            continue
    return terms

# ---- Relevant Lexical Resources ----
causal_frames = ["Causation", "Cause_to_start", "Preventing", "Risk", "Threaten", "Danger"]
causal_verbs = get_lexical_units_from_frames(causal_frames)

fear_keywords = {"danger", "threat", "risky", "harm", "catastrophe", "crisis", "ruin", "fear", "worse", "bad", "fatal", "negative", "die", "death"}
preventive_keywords = {"prevent", "avoid", "stop", "ban", "rescue", "save", "protect"}

urgency_keywords = {"immediately", "soon", "before it's too late", "critical", "urgent", "suddenly", "unexpectedly"}

possibility_terms = {"possible", "possibility", "likely", "likelihood", "chance", "probability", "conceivable", "potential", "can", "could", "might", "may", "able"}


def detect_fear_appeal_question(question):
    doc = nlp(question)
    score = 0
    explanations = []

    if any(tok.lemma_.lower() in fear_keywords for tok in doc):
        explanations.append("✓ Fear-related keyword detected (e.g., 'threat', 'danger')")
        score += 3

    if any(tok.lemma_.lower() in preventive_keywords for tok in doc):
        explanations.append("✓ Preventive action verb detected (e.g., 'prevent', 'stop')")
        score += 2

    if any(tok.lemma_ in causal_verbs for tok in doc if tok.pos_ == "VERB"):
        explanations.append("✓ Causal/preventive verb from FrameNet detected")
        score += 2

    if any(tok.text.lower() in {"if", "unless"} for tok in doc):
        explanations.append("✓ Conditional clause found (e.g., 'if', 'unless')")
        score += 1

    if any(is_fear_related(tok) for tok in doc if tok.pos_ in {"NOUN", "VERB", "ADJ"}):
        explanations.append("✓ Semantic fear-related concept detected via WordNet")
        score += 2

    if any(phrase in question.lower() for phrase in urgency_keywords):
        explanations.append("✓ Urgency marker detected (e.g., 'immediately', 'before it's too late')")
        score += 1

    if any(tok.lemma_ in possibility_terms for tok in doc):
        explanations.append("✓ Possibility-related term detected (e.g., 'possible', 'feasible', 'chance')")
        score += 1

    score = min(score, 10)
    label = "Strong Fear Appeal" if score >= 7 else "Weak/Partial Fear Appeal" if score >= 4 else "Not Fear Appeal"
    return label, score, explanations

## Final method for preprocessing and filtering for all types

In [ ]:
def classify_schema(row):
    question = row["question"]
    results = []

    is_critical = False

    _, cte_score, _ = detect_cause_to_effect(question)
    _, expert_score, _ = detect_expert_opinion(question)
    _, analogy_score, _ = detect_analogy_question(question)
    _, fear_score, _ = detect_fear_appeal_question(question)

    if cte_score >= 7:
      is_critical = True

    if expert_score >= 7:
      is_critical = True

    if analogy_score >= 7:
      is_critical = True

    if fear_score >= 7:
       is_critical = True

    return pd.Series({
        "is_Critical": is_critical,
        "CauseToEffect": cte_score,
        "ExpertOpinion": expert_score,
        "Analogy": analogy_score,
        "FearAppeal": fear_score,
    })


filtered_df[["is_Critical","CauseToEffect", "ExpertOpinion", "Analogy", "FearAppeal"]] = filtered_df.apply(classify_schema, axis=1)
filtered_df

### final check: how many questions per category ?

In [ ]:
value_counts = filtered_df['is_Critical'].value_counts(dropna=False)
print(value_counts)

columns = ["CauseToEffect", "ExpertOpinion", "Analogy", "FearAppeal"]

value_counts_schema = filtered_df[columns].apply(pd.Series.value_counts, dropna=False)
print(value_counts_schema)

In [ ]:
value_counts_schema = filtered_df[columns].apply(pd.Series.value_counts, dropna=False)

per_column_gte_7 = value_counts_schema[value_counts_schema.index >= 7].sum()

print("Entries with score >= 7 per column:")
print(per_column_gte_7)

## Save and filter datasets

In [ ]:
filtered_df.to_json(f'/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/scored/processed_{chunk}.json', index=False)

In [ ]:
pro_chunk_I_df = pd.read_json('/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/scored/processed_train_chunk_I.json')
pro_chunk_II_df = pd.read_json('/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/scored/processed_train_chunk_II.json')
pro_chunk_III_df = pd.read_json('/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/scored/processed_train_chunk_III.json')

combined_df = pd.concat([pro_chunk_I_df, pro_chunk_II_df, pro_chunk_III_df], ignore_index=True)

schema_cols = ['CauseToEffect', 'ExpertOpinion', 'Analogy', 'FearAppeal']

long_df = combined_df.melt(
    id_vars=['context', 'question'],
    value_vars=schema_cols,
    var_name='schema',
    value_name='score'
)

qualified = long_df[long_df['score'] >= 7].copy()

qualified = qualified.drop(columns=['score'])

mask_no_schema = combined_df[schema_cols].max(axis=1) < 7

no_schema_df = combined_df.loc[mask_no_schema, ['context', 'question']].copy()
no_schema_df['schema'] = ''

final_df = pd.concat([qualified, no_schema_df], ignore_index=True)

final_df.shape

In [ ]:
df_to_save = combined_df.copy()

df_to_save['id'] = range(1, len(df_to_save) + 1)

columns_to_save = [
    'id',
    'context',
    'question',
    'context_token_len',
    'is_Critical',
    'CauseToEffect',
    'Analogy',
    'ExpertOpinion',
    'FearAppeal'
]

df_to_save[columns_to_save].to_json('/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/scored/processed_train_data.json', orient='records', indent=2)

In [ ]:
final_df = final_df.reset_index(drop=True)
final_df['id'] = final_df.index + 1

final_df = final_df[['id', 'context', 'question', 'schema']]

final_df.to_json('/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/Categorised/categoriesed_train_data.json', orient='records', indent=2)

In [ ]:
filtered_df = final_df[final_df['schema'] != ''].copy()

filtered_df = filtered_df.reset_index(drop=True)
filtered_df['id'] = filtered_df.index + 1

filtered_df.to_json('/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/Categorised/categoriesed_filtered_train_data.json', orient='records', indent=2)

In [11]:
full_df = pd.read_json('/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/scored/processed_train_data.json', orient='records')

score_columns = ['CauseToEffect', 'Analogy', 'ExpertOpinion', 'FearAppeal']
filtered_df = full_df[full_df[score_columns].gt(4).any(axis=1)]

filtered_df.to_json('/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/score_filtered/score_4_train_data.json', orient='records', indent=2)

(18902, 9)

## analyse dataset

In [17]:
df = pd.read_json('/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/scored/processed_train_data.json', orient='records')
df.shape

(80985, 9)

In [20]:
score_columns = ['CauseToEffect', 'Analogy', 'ExpertOpinion', 'FearAppeal']

score_counts = pd.DataFrame()

for col in score_columns:
    counts = df[col].value_counts().sort_index()
    score_counts[col] = counts

score_counts = score_counts.fillna(0).astype(int)

print(score_counts)

               CauseToEffect  Analogy  ExpertOpinion  FearAppeal
CauseToEffect                                                   
0.0                    45081     4466          49444         650
1.0                      358     3178           2423          32
2.0                    15451    15157          12691       55079
3.0                     1093    27914           7266       11209
3.5                     4427     9877             83           0
4.0                     5360     6202           5037        8837
4.5                     1205     3193             43           0
5.0                     5020      807           1915        3845
5.5                      844      482             19           0
6.0                      642     1758            620         745
6.5                      212      732              6           0
7.0                       70      686            208         433
7.5                       26      362              4           0
8.5                      

## Create Dataset for DPO Traing

In [ ]:
import json
import pandas as pd

# Load the existing JSON dataset
input_file = 'Data/Processed/processed_train_data.json'
output_file = 'Data/Processed/processed_train_data_for_dpo.json'


In [ ]:
with open(input_file, 'r') as f:
    data = json.load(f)

# Prepare the new data format in conversational style
conversational_data = []
for entry in data:
    conversational_entry = {
        "prompt": [{"role": "user", "content": "You are an AI assistant to help:"}],
        "completion": [{"role": "assistant", "content": entry.get("question", "")}],
        "label": entry.get("is_Critical", False)
    }
    conversational_data.append(conversational_entry)

# Convert to DataFrame for a more structured view
conversational_df = pd.DataFrame(conversational_data)

In [ ]:
def balance_labels(df):
    true_df = df[df['label'] == True]
    false_df = df[df['label'] == False]
    min_size = min(len(true_df), len(false_df))
    balanced_df = pd.concat([true_df.sample(min_size, random_state=42), false_df.sample(min_size, random_state=42)])
    return balanced_df.reset_index(drop=True)

In [ ]:
# Balance the dataset
balanced_df = balance_labels(conversational_df)
# Save the reformatted data to a new JSON file
balanced_df.to_json(output_file, orient='records', lines=True)

print(f"Conversational style dataset saved to: {output_file}")


Conversational style dataset saved to: Data/Processed/processed_train_data_for_dpo.json


## Git

In [ ]:
!git config --global user.name "Showcas"
!git config --global user.email "cedric.bohni@gmx.de"


commit_message = f"Improved Preprocessing file"
!git add .
!git commit -m "{commit_message}"
!git push